In [22]:
import os
print(os.environ["BENTOML_HOME"])

/bentoml/


In [3]:
import pandas as pd
import numpy as np

from sklearn import svm
from sklearn import datasets

import bentoml
from bentoml.io import NumpyNdarray, PandasDataFrame

## load sample dataset

In [4]:
iris = datasets.load_iris()

In [20]:
print(f"input data shape: {iris.data.shape}")
print(f"target data shape: {iris.target.shape}")
print(f"num classes: {len(pd.unique(iris.target))}")

input data shape: (150, 4)
target data shape: (150,)
num classes: 3


In [21]:
'''
check out the dataset description
'''
print(iris.DESCR)

.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

    :Number of Instances: 150 (50 in each of three classes)
    :Number of Attributes: 4 numeric, predictive attributes and the class
    :Attribute Information:
        - sepal length in cm
        - sepal width in cm
        - petal length in cm
        - petal width in cm
        - class:
                - Iris-Setosa
                - Iris-Versicolour
                - Iris-Virginica
                
    :Summary Statistics:

    ============== ==== ==== ======= ===== ====================
                    Min  Max   Mean    SD   Class Correlation
    ============== ==== ==== ======= ===== ====================
    sepal length:   4.3  7.9   5.84   0.83    0.7826
    sepal width:    2.0  4.4   3.05   0.43   -0.4194
    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
    petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)
    ============== ==== ==== ======= ===== ===========

## train classifier

In [6]:
X, y = iris.data, iris.target

In [7]:
clf = svm.SVC(gamma="scale")

In [8]:
clf.fit(X, y)

SVC()

## save model
By running command below in the container, you can check out & manage the saved models.  
```bash
$ bentoml models list
 Tag                        Module           Size      Creation Time
 iris_clf:iuf2xbcbl2pweasc  bentoml.sklearn  5.98 KiB  2022-10-01 07:54:25
```
    
Also, you can see the directory structure of ${BENTOML_HOME} by runnning command below.  
```bash
$ tree /bentoml/

bentoml/
├── bentos
├── client_id
└── models
    └── iris_clf
        ├── iuf2xbcbl2pweasc
        │   ├── model.yaml
        │   └── saved_model.pkl
        └── latest
```

In [9]:
bentoml.sklearn.save_model(
    name="iris_clf",
    model=clf,
)

Model(tag="iris_clf:iuf2xbcbl2pweasc", path="/bentoml/models/iris_clf/iuf2xbcbl2pweasc/")

## define services

In [26]:
iris_clf_runner = bentoml.sklearn.get("iris_clf:latest").to_runner()

In [28]:
svc = bentoml.Service("iris_classifier", runners=[iris_clf_runner])

In [31]:
@svc.api(input=PandasDataFrame(), output=NumpyNdarray())
def predict(input_series: pd.DataFrame) -> np.ndarray:
    result = iris_clf_runner.predict.run(input_series)
    return result

## environment
Bentoml version 1.x.x no longer depends on `@bentoml.api` decorator.  
Environment settings and service dependencies are defined in the `bentofile.yaml` file in the ${BENTOML_HOME} directory.  
The contents are used to specify the bentoml build opations when building bentos.  


```bash
$ cat /bentoml/models/iris_clf/iuf2xbcbl2pweasc/model.yaml

name: iris_clf
version: iuf2xbcbl2pweasc
module: bentoml.sklearn
labels: {}
options: {}
metadata: {}
context:
  framework_name: sklearn
  framework_versions:
    scikit-learn: 1.1.2
  bentoml_version: 1.0.6
  python_version: 3.10.6
signatures:
  predict:
    batchable: false
api_version: v1
creation_time: '2022-10-01T07:54:25.948067+00:00'
```